Error codes 400-499 means that there wasn an error from the client

In [1]:
#use httpexception
import nest_asyncio
import uvicorn
nest_asyncio.apply()



from fastapi import FastAPI,HTTPException, status
app = FastAPI()


items  = {"foo": "this is the first item"}

@app.get('/items/{item_id}')
async def read_item(item_id: str):
    if item_id not in items:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Item Not found")
    return {"item": items[item_id]}
    

uvicorn.run(app, port=8000)



INFO:     Started server process [9161]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:33680 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:43882 - "GET /items/foo HTTP/1.1" 200 OK
INFO:     127.0.0.1:37720 - "GET /items/bar HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9161]


In [3]:
#Adding custom headers


from fastapi import FastAPI, HTTPException

app = FastAPI()


items = {'foo': 'The Foo Wrestlers'}


@app.get('/items/{item_id}')
async def read_item_header(item_id: str):
    if item_id not in items:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            description= "Item not found",
            headers={"X-error": "There goes my error"},
        )
    
    return {"item": items[item_id]}


uvicorn.run(app, port=8000)

INFO:     Started server process [9161]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56442 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56442 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56442 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:40070 - "GET /items/foo HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9161]


Install custom exception handlers

In [4]:
#You can add custom exception handlers with the sam exception utilities from starlette
#Lets say you have a custom exception UnivornException that you might raise.

#And you want to handle this exception globally with FastAPI


#You could add a custom exception handler with @app.exception_handler():

from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse


class UnicornException(Exception):
    def __init__(self, name:str):
        self.name = name


app = FastAPI()


@app.exception_handler(UnicornException)
async def unicorn_exception_handler(request: Request, exc: UnicornException):
    return JSONResponse(
        status_code=status.HTTP_418_IM_A_TEAPOT,
        content={"message": f"oops! {exc.name} did something. There goes a rainbow..."},
    )


@app.get("/unicorns/{name}")
async def read_unicorn(name: str):
    if name == "yolo":
        raise UnicornException(name=name)
    return {'unicorn_name': name}


uvicorn.run(app, port=8000)

INFO:     Started server process [9161]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:45490 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:45490 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:45288 - "GET /unicorns/yolo HTTP/1.1" 418 I'm a Teapot


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9161]


Adding custom exception handlers

In [1]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse


class UnicornException(Exception):
    def __init__(self, name: str):
        self.name = name

app = FastAPI()


@app.exception_handler(UnicornException)
async def unicorn_exception_handler(request: Request, exc: UnicornException):
    return JSONResponse(
        status_code=418,
        content={'message': f'Oops! {exc.name} did something. There goes a rainbow ...'},
    )

@app.get('/unicorns/{name}')
async def read_unicorn(name: str):
    if name == 'yolo':
        raise UnicornException(name=name)
    return {"unicorn_name": name}



uvicorn.run(app, port=8000)

INFO:     Started server process [7963]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:43022 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:43022 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:39286 - "GET /unicorns/yolo HTTP/1.1" 418 I'm a Teapot
INFO:     127.0.0.1:39288 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:39288 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50028 - "GET /unicorns/yolo HTTP/1.1" 418 I'm a Teapot


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7963]


Override the default exception handlers

In [ ]:
#The following returns a plaintext response

from fastapi import FastAPI, HTTPException
from fastapi.exceptions import RequestValidationError
from fastapi.responses import PlainTextResponse
from starlette.exceptions import HTTPException as StarletteHttpException


app = FastAPI()


@app.exception_handler(StarletteHttpException)
async def http_exception_handler(request, exc):
    return PlainTextResponse(str(exc.detail), status_code = exc.status_code)



@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request, exc):
    return PlainTextResponse(str(exc), status_code=400)


@app.get('/items/{item_id}')
async def read_item(item_id: int):
    if item_id == 3:
        raise HTTPException(status_code=418, detail='Nope! I dont like 3.')
    return {'item_id': item_id}


uvicorn.run(app, port=8000)